## Summary

Calculate features using [ProtBert](https://github.com/agemagician/ProtTrans).

```bash
export NOTEBOOK_PATH="$(realpath 31_run_rosetta_ddg.ipynb)"

export DATASET_NAME="cagi6-sherloc"
export DATASET_PATH="30_cagi6_sherloc/input-data-gby-protein.parquet"
export ORIGINAL_ARRAY_TASK_COUNT=4182

sbatch --export DATASET_NAME,DATASET_PATH,NOTEBOOK_PATH,ORIGINAL_ARRAY_TASK_COUNT --array=1-1 --time 3:00:00 --mem=0 --exclusive ../scripts/run_notebook_cpu.sh


# Niagara
sbatch --export DATASET_NAME,DATASET_PATH,NOTEBOOK_PATH,ORIGINAL_ARRAY_TASK_COUNT --array=1-4182 --time 3:00:00 --mem=0 --exclusive ../scripts/run_notebook_cpu.sh
```

---

## Imports

In [1]:
import concurrent.futures
import itertools
import os
import re
import socket
import subprocess
import sys
import tempfile
from pathlib import Path

import kmbio
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from elaspic2.plugins.rosetta_ddg import RosettaDDG
from kmbio import PDB
from tqdm.notebook import tqdm

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-q_ssaop3 because the default path (/home/p/pmkim/strokach/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


## Parameters

In [2]:
NOTEBOOK_DIR = Path("31_run_rosetta_ddg").resolve()
NOTEBOOK_DIR.mkdir(exist_ok=True)

NOTEBOOK_DIR

PosixPath('/scratch/p/pmkim/strokach/workspace/elaspic2-cagi6/notebooks/31_run_rosetta_ddg')

In [3]:
if (slurm_tmpdir := os.getenv("SLURM_TMPDIR")) is not None:
    os.environ["TMPDIR"] = slurm_tmpdir
    
print(tempfile.gettempdir())

/tmp


In [4]:
if "scinet" in socket.gethostname():
    CPU_COUNT = 40
else:
    CPU_COUNT = max(1, len(os.sched_getaffinity(0)))

# CPU_COUNT = max(1, CPU_COUNT // 2)

CPU_COUNT

40

In [5]:
DATASET_NAME = os.getenv("DATASET_NAME")
DATASET_PATH = os.getenv("DATASET_PATH")
TASK_ID = os.getenv("SLURM_ARRAY_TASK_ID")
TASK_COUNT = os.getenv("ORIGINAL_ARRAY_TASK_COUNT") or os.getenv("SLURM_ARRAY_TASK_COUNT")

TASK_ID = int(TASK_ID) if TASK_ID is not None else None
TASK_COUNT = int(TASK_COUNT) if TASK_COUNT is not None else None

DATASET_NAME, DATASET_PATH, TASK_ID, TASK_COUNT

(None, None, None, None)

In [6]:
DEBUG = TASK_ID is None

if DEBUG:
    DATASET_NAME = "cagi6-sherloc"
    DATASET_PATH = str(
        NOTEBOOK_DIR.parent.joinpath("30_cagi6_sherloc", "input-data-gby-protein.parquet")
    )
    TASK_ID = 1
    TASK_COUNT = 4182
else:
    assert DATASET_NAME is not None
    assert DATASET_PATH is not None
    assert TASK_ID is not None
    assert TASK_COUNT is not None
    DATASET_PATH = Path(DATASET_PATH).expanduser().resolve()

DATASET_NAME, DATASET_PATH, TASK_ID, TASK_COUNT

('cagi6-sherloc',
 '/scratch/p/pmkim/strokach/workspace/elaspic2-cagi6/notebooks/30_cagi6_sherloc/input-data-gby-protein.parquet',
 1,
 4182)

## Workspace

### Load data

In [7]:
pfile = pq.ParquetFile(DATASET_PATH)

pfile.num_row_groups

4182

In [8]:
assert TASK_COUNT == pfile.num_row_groups

In [9]:
input_df = pfile.read_row_group(TASK_ID - 1).to_pandas(integer_object_nulls=True)

In [10]:
display(input_df.head(2))
print(len(input_df))

,protein_id,mutation_id,mutation,effect,sequence,structure,alignment
0,P26678,"[NM_002667.3:c.25C>T, NM_002667.3:c.22A>C, NM_...","[R9C, T8P, F32S, P21T, R25C, R25H, I38T, F32S,...","[Pathogenic, Uncertain significance, None, Unc...",MEKVQYLTRSAIRRASTIEMPQQARQKLQNLFINFCLILICLLLIC...,HEADER ...,"[>101\n, MEKVQYLTRSAIRRASTIEMPQQARQKLQNLFINFCL..."


1


In [11]:
tup = next(input_df.itertuples(index=False))

iterable_fields = []
for field in tup._fields:
    try:
        if len(getattr(tup, field)) == len(tup.mutation):
            iterable_fields.append(field)
    except TypeError:
        pass

iterable_fields

['mutation_id', 'mutation', 'effect']

In [12]:
if DEBUG:
    for field in iterable_fields:
        input_df[field] = input_df[field].str[:3]

### Score mutations

In [13]:
def validate_mutation(mutation):
    aa = "GVALICMFWPDESTYQNKRH"
    if re.search(f"^[{aa}][1-9]+[0-9]*[{aa}]$", mutation) is None:
        print(f"Skipping mutation {mutation} because it appears to be malformed.")
        return False

    if mutation[0] == mutation[-1]:
        print(
            f"Skipping mutation {mutation} because the wildtype and mutant residues are the same."
        )
        return False

    return True

In [14]:
def worker(mutation, data):
    if not validate_mutation(mutation):
        return None

    try:
        results = RosettaDDG.analyze_mutation(f"A_{mutation}", data)
    except Exception as error:
        print(f"{error!r}")
        return None
    else:
        results = {f"rosetta_{key}": value for key, value in results.items()}
        return results

In [15]:
results = []
for tup in tqdm(input_df.itertuples(index=False), total=len(input_df)):
    assert all([(len(getattr(tup, field)) == len(tup.mutation)) for field in iterable_fields])

    with tempfile.NamedTemporaryFile(suffix=".pdb") as tmp_file:
        with open(tmp_file.name, "wt") as fout:
            fout.write(tup.structure)
        data = RosettaDDG.build(
            tmp_file.name, protocol="cartesian_ddg", energy_function="beta_nov16_cart", interface=0
        )

    with concurrent.futures.ProcessPoolExecutor(CPU_COUNT) as pool:
        result_list = list(
            tqdm(
                pool.map(worker, tup.mutation, itertools.repeat(data)),
                total=len(tup.mutation),
                leave=False,
            )
        )

    for mutation_idx, (mutation, result) in enumerate(zip(tup.mutation, result_list)):
        if result is None:
            continue

        results.append(
            {
                "protein_id": tup.protein_id,
                "mutation": mutation,
            }
            | {field: getattr(tup, field)[mutation_idx] for field in iterable_fields}
            | result
        )

results_df = pd.DataFrame(results)

/opt/conda/lib/python3.9/site-packages/elaspic2/plugins/rosetta_ddg/functions.py:109: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  return df.mean().to_dict()
/opt/conda/lib/python3.9/site-packages/elaspic2/plugins/rosetta_ddg/functions.py:109: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  return df.mean().to_dict()
/opt/conda/lib/python3.9/site-packages/elaspic2/plugins/rosetta_ddg/functions.py:109: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  return df.mean().to_dict()


In [16]:
display(results_df.head(2))
print(len(results_df))

,protein_id,mutation,mutation_id,effect,rosetta_cart_bonded_wt,rosetta_rama_prepro_wt,rosetta_ref_wt,rosetta_hxl_tors_wt,rosetta_p_aa_pp_wt,rosetta_fa_dun_semi_wt,...,rosetta_lk_ball_bridge_change,rosetta_lk_ball_iso_change,rosetta_lk_ball_change,rosetta_fa_intra_sol_xover4_change,rosetta_fa_intra_rep_xover4_change,rosetta_fa_intra_atr_xover4_change,rosetta_fa_sol_change,rosetta_fa_rep_change,rosetta_fa_atr_change,rosetta_dg_change
0,P26678,R9C,NM_002667.3:c.25C>T,Pathogenic,34.962667,18.085667,13.08,5.892667,-4.872667,33.824667,...,0.047667,1.492667,-0.024333,-0.074667,-0.208667,0.266,-4.181000,0.037667,2.272333,3.887
1,P26678,T8P,NM_002667.3:c.22A>C,Uncertain significance,35.520333,17.844667,13.08,9.764333,-5.050333,33.552667,...,0.003667,0.933333,0.089333,-0.140333,-0.133667,0.435,-1.526333,6.797000,-1.945667,19.596


3


### Save results

In [17]:
output_file = NOTEBOOK_DIR.joinpath(DATASET_NAME, f"shard-{TASK_ID}-of-{TASK_COUNT}.parquet")
output_file.parent.mkdir(exist_ok=True)

output_file

PosixPath('/scratch/p/pmkim/strokach/workspace/elaspic2-cagi6/notebooks/31_run_rosetta_ddg/cagi6-sherloc/shard-1-of-4182.parquet')

In [18]:
if not DEBUG:
    pq.write_table(pa.Table.from_pandas(results_df, preserve_index=False), output_file)